In [1]:
!pip install fastai
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.2/29.2 MB 28.1 MB/s eta 0:00:00


In [2]:
from fastai.tabular.all import *
from fastai.text.all import *
from rdkit.Chem import Descriptors, MolFromSmiles

In [3]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/bioactivity-data/PPARg_bioactivity_data_nM.csv
/kaggle/input/bioactivity-data/PPARg_bioactivity_data_no_missing_data_values .csv


In [4]:
biodata = pd.read_csv('/kaggle/input/bioactivity-data/PPARg_bioactivity_data_nM.csv')

# se på data

In [5]:
pd.set_option('display.max_columns', None)

In [6]:
biodata.nunique()

activity_comment               50
activity_id                  1445
activity_properties             3
assay_chembl_id               127
assay_description             122
assay_type                      3
assay_variant_accession         0
assay_variant_mutation          0
bao_endpoint                    2
bao_format                      3
bao_label                       3
canonical_smiles             1247
data_validity_comment           1
data_validity_description       1
document_chembl_id             99
document_journal               13
document_year                  19
ligand_efficiency             673
molecule_chembl_id           1247
molecule_pref_name             28
parent_molecule_chembl_id    1247
pchembl_value                 341
potential_duplicate             2
qudt_units                      1
record_id                    1378
relation                        4
src_id                          2
standard_flag                   1
standard_relation               4
standard_text_

In [7]:
unik = biodata.nunique()
unikkol = unik[unik == 1].index
print(unikkol.tolist())

['data_validity_comment', 'data_validity_description', 'qudt_units', 'standard_flag', 'standard_type', 'standard_units', 'target_chembl_id', 'target_organism', 'target_pref_name', 'target_tax_id', 'type', 'units', 'uo_units']


In [8]:
biodata.head(1000)

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,NaN,32233,[],CHEMBL658541,In vitro agonist efficacy against PPAR gamma along with 100 nM BRL-49653,F,NaN,NaN,BAO_0000188,BAO_0000019,assay format,CC(/C=C/C(F)=C(/C)c1cc(C(C)(C)C)cc(C(C)(C)C)c1OCC(F)F)=C\C(=O)O,NaN,NaN,CHEMBL1136440,Bioorg. Med. Chem. Lett.,2003,NaN,CHEMBL111217,NaN,CHEMBL111217,7.20,0,http://www.openphacts.org/units/Nanomolar,251326,=,1,1,=,NaN,EC50,nM,NaN,63.0,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,NaN,NaN,EC50,nM,UO_0000065,NaN,63.0
1,NaN,32234,[],CHEMBL656170,In vitro agonistic activity against PPAR gamma along with 100 nM BRL-49653,F,NaN,NaN,BAO_0000188,BAO_0000019,assay format,CC(/C=C/C(F)=C(/C)c1cc(C(C)(C)C)cc(C(C)(C)C)c1OCC(F)F)=C\C(=O)O,NaN,NaN,CHEMBL1136440,Bioorg. Med. Chem. Lett.,2003,NaN,CHEMBL111217,NaN,CHEMBL111217,8.19,0,http://www.openphacts.org/units/Nanomolar,251326,=,1,1,=,NaN,EC50,nM,NaN,6.4,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,NaN,NaN,EC50,nM,UO_0000065,NaN,6.4
2,NaN,33386,[],CHEMBL658541,In vitro agonist efficacy against PPAR gamma along with 100 nM BRL-49653,F,NaN,NaN,BAO_0000188,BAO_0000019,assay format,CCC(C)(C)c1cc(/C(C)=C(F)\C=C\C(C)=C\C(=O)O)c(OCC(F)F)c(C(C)(C)CC)c1,NaN,NaN,CHEMBL1136440,Bioorg. Med. Chem. Lett.,2003,NaN,CHEMBL130231,NaN,CHEMBL130231,7.29,0,http://www.openphacts.org/units/Nanomolar,251310,=,1,1,=,NaN,EC50,nM,NaN,51.0,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,NaN,NaN,EC50,nM,UO_0000065,NaN,51.0
3,NaN,33387,[],CHEMBL656170,In vitro agonistic activity against PPAR gamma along with 100 nM BRL-49653,F,NaN,NaN,BAO_0000188,BAO_0000019,assay format,CCC(C)(C)c1cc(/C(C)=C(F)\C=C\C(C)=C\C(=O)O)c(OCC(F)F)c(C(C)(C)CC)c1,NaN,NaN,CHEMBL1136440,Bioorg. Med. Chem. Lett.,2003,NaN,CHEMBL130231,NaN,CHEMBL130231,8.57,0,http://www.openphacts.org/units/Nanomolar,251310,=,1,1,=,NaN,EC50,nM,NaN,2.7,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,NaN,NaN,EC50,nM,UO_0000065,NaN,2.7
4,NaN,35892,[],CHEMBL658541,In vitro agonist efficacy against PPAR gamma along with 100 nM BRL-49653,F,NaN,NaN,BAO_0000188,BAO_0000019,assay format,C/C(=C/C=C/C(C)=C/C(=O)O)c1cc(-c2cc(F)cc(F)c2)cc(C(C)C)c1OCC(F)F,NaN,NaN,CHEMBL1136440,Bioorg. Med. Chem. Lett.,2003,NaN,CHEMBL337393,NaN,CHEMBL337393,7.40,0,http://www.openphacts.org/units/Nanomolar,251312,=,1,1,=,NaN,EC50,nM,NaN,40.0,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,NaN,NaN,EC50,nM,UO_0000065,NaN,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,15042190,[],CHEMBL3369562,Agonist activity at human PPARgamma,B,NaN,NaN,BAO_0000188,BAO_0000357,single protein format,CCCOc1ccc(C[C@@H](Cc2ccccc2)C(=O)O)cc1CNC(=O)c1ccc(N2CC(C)(C)CC(C)(C)C2)cc1,NaN,NaN,CHEMBL3351353,Bioorg. Med. Chem. Lett.,2014,"{'bei': '15.17', 'le': '0.28', 'lle': '1.51', 'sei': '10.98'}",CHEMBL3317863,NaN,CHEMBL3317863,8.66,0,http://www.openphacts.org/units/Nanomolar,2246640,=,1,1,=,NaN,EC50,nM,NaN,2.2,CHEMBL235,Homo sapiens,Peroxisome proliferator-activated receptor gamma,9606,NaN,NaN,EC50,nM,UO_0000065,NaN,2.2
996,NaN,15042191,[],CHEMBL3369562,Agonist activity at human PPARgamma,B,NaN,NaN,BAO_0000188,BAO_00003

In [9]:
biodata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1445 entries, 0 to 1444
Data columns (total 45 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   activity_comment           50 non-null     float64
 1   activity_id                1445 non-null   int64  
 2   activity_properties        1445 non-null   object 
 3   assay_chembl_id            1445 non-null   object 
 4   assay_description          1445 non-null   object 
 5   assay_type                 1445 non-null   object 
 6   assay_variant_accession    0 non-null      float64
 7   assay_variant_mutation     0 non-null      float64
 8   bao_endpoint               1445 non-null   object 
 9   bao_format                 1445 non-null   object 
 10  bao_label                  1445 non-null   object 
 11  canonical_smiles           1445 non-null   object 
 12  data_validity_comment      4 non-null      object 
 13  data_validity_description  4 non-null      objec

In [10]:
len(biodata)

1445

In [11]:
biodata.isna().sum()

activity_comment             1395
activity_id                     0
activity_properties             0
assay_chembl_id                 0
assay_description               0
assay_type                      0
assay_variant_accession      1445
assay_variant_mutation       1445
bao_endpoint                    0
bao_format                      0
bao_label                       0
canonical_smiles                0
data_validity_comment        1441
data_validity_description    1441
document_chembl_id              0
document_journal               50
document_year                   0
ligand_efficiency             748
molecule_chembl_id              0
molecule_pref_name           1336
parent_molecule_chembl_id       0
pchembl_value                 172
potential_duplicate             0
qudt_units                     61
record_id                       0
relation                        0
src_id                          0
standard_flag                   0
standard_relation               0
standard_text_

# prepare data


**dropper dårlig data**

In [12]:
#sjekker om radene har de samme verdiene, fordi da er det ikke vits i å ha de med
unik = biodata.nunique()
unikkol = unik[unik == 1].index
print(unikkol.tolist())

['data_validity_comment', 'data_validity_description', 'qudt_units', 'standard_flag', 'standard_type', 'standard_units', 'target_chembl_id', 'target_organism', 'target_pref_name', 'target_tax_id', 'type', 'units', 'uo_units']


In [13]:
#dropper de har alle radene som er like 
biodata.drop(['data_validity_comment', 'data_validity_description', 'qudt_units', 'standard_flag', 'standard_type', 'standard_units', 'target_chembl_id', 
              'target_organism', 'target_pref_name', 'target_tax_id', 'type', 'units', 'uo_units'], axis=1, inplace=True)

In [14]:
#Jeg dropper featurså som er like, feks value og standard_value: da dropper jeg standardvalue. og de som er urelevant sånn som id
biodata.drop(["document_year",  "activity_id", "assay_chembl_id", "bao_label", "document_chembl_id", "molecule_chembl_id",
             "parent_molecule_chembl_id", "record_id", "standard_value", "standard_relation", "relation", "src_id", "activity_comment"],axis=1, inplace=True)


In [15]:
#dropper de med for mange manglende verdier.
biodata.drop(["assay_variant_accession", "assay_variant_mutation", "molecule_pref_name","standard_text_value", "standard_upper_value", "toid", "upper_value", "ligand_efficiency", 
              "text_value"],axis=1, inplace=True)


In [16]:
#dropper de kolonnene som har manglende verdier. og jeg skal bruke pchemble_value som target
biodata = biodata.dropna(subset=["pchembl_value", "document_journal", "assay_description"], axis=0)

In [17]:
#få representert canonical_smiles
#kopiert
def smiles_to_descriptors(smiles):
    mol = MolFromSmiles(smiles)
    return [Descriptors.MolWt(mol), Descriptors.NumHAcceptors(mol), Descriptors.NumHDonors(mol)]

descriptors = biodata["canonical_smiles"].apply(smiles_to_descriptors)

In [18]:
#lage et nytt dataframe med molkyl
data_descriptors = pd.DataFrame(np.vstack(descriptors), columns=["MolWt", "NumHAcceptors", "NumHDonors"])

In [19]:
#bruke d nye og droppe det gamle
biodata = pd.concat([biodata, data_descriptors], axis=1)
biodata = biodata.drop("canonical_smiles", axis=1)

**tokenized**

In [20]:
#tokenized for kommentaene
text_dls = TextDataLoaders.from_df(biodata, text_col='assay_description', is_lm=False, valid_pct=0.2)

In [21]:
#prøvte å kombinere text og tabular men får det ikke til....
#db = DataBlock(blocks=(TextBlock(vocab=text_dls.vocab), CategoryBlock(vocab=cat_cols), ContinBlock, RegressionBlock),
#               get_x=[ColReader('assay_description', pref='text_'), ColReader(cat_cols), ColReader(cont_cols)],
#               get_y=[ColReader('pchembl_value')],
#               splitter=RandomSplitter(valid_pct=0.2))

In [22]:
lm_dls = TextDataLoaders.from_df(biodata, text_col='assay_description', is_lm=True, valid_pct=0.2)

# lager en modell
lm_learn = language_model_learner(lm_dls, AWD_LSTM, drop_mult=0.3, metrics=[accuracy, Perplexity()])
lm_learn.fit_one_cycle(1, 2e-2)
lm_learn.unfreeze()
lm_learn.fit_one_cycle(5, 2e-3)

#lagrer
lm_learn.save_encoder('finetuned')

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,5.520812,3.456732,0.362591,31.713177,00:25


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.032201,2.229276,0.556049,9.293138,00:36
1,2.528548,1.516990,0.671776,4.558482,00:35
2,2.085282,1.084074,0.738646,2.956700,00:35
3,1.774547,0.906375,0.783183,2.475334,00:35
4,1.549777,0.869802,0.792515,2.386437,00:35


**Transforms**

In [23]:
#droppper assay_description siden den ødelegger når jeg ikke klarer å kombinere 
#biodata.drop("assay_description",axis=1, inplace=True)

In [24]:
#hva er katoring og kva er kontinuerlig
cat_cols = ['assay_description','assay_type', 'bao_endpoint', 'bao_format', 'document_journal']
#cat_cols = ['assay_type', 'bao_endpoint', 'bao_format', 'document_journal']
cont_cols = ['value', 'MolWt', 'NumHAcceptors', 'NumHDonors']

In [25]:
cat_cols

['assay_description',
 'assay_type',
 'bao_endpoint',
 'bao_format',
 'document_journal']

In [26]:
cont_cols

['value', 'MolWt', 'NumHAcceptors', 'NumHDonors']

In [27]:
#preprosessor
procs = [Categorify, Normalize]

In [28]:
splits = RandomSplitter(valid_pct=0.2)(range_of(biodata))

In [29]:
#tabularpandas
to = TabularPandas(biodata, procs, cat_cols, cont_cols, y_names='pchembl_value', splits=splits)

In [30]:
print(to.train.xs.head())

      assay_description  assay_type  bao_endpoint  bao_format  \
394                  74           2             2           2   
1276                 60           2             2           2   
744                  17           3             2           1   
619                  52           3             2           2   
608                  32           3             2           2   

      document_journal     value     MolWt  NumHAcceptors  NumHDonors  
394                  5  4.502992  0.776141      -0.821048    1.338489  
1276                 3 -0.363920       NaN            NaN         NaN  
744                  6  0.056762  0.378343       1.848558   -0.498394  
619                  6 -0.398122  0.917829       0.513755   -0.498394  
608                  6  0.227771  1.048916       0.513755   -0.498394  


In [31]:
len(to.train),len(to.valid)

(1123, 280)

In [32]:
to.show()

,assay_description,assay_type,bao_endpoint,bao_format,document_journal,value,MolWt,NumHAcceptors,NumHDonors,pchembl_value
394,Effect on PPARgamma transactivation activity in HEK293 cells,B,BAO_0000188,BAO_0000219,Bioorg. Med. Chem.,8600.000000,519.491028,4.0,2.0,5.07
1276,Agonist activity at recombinant human GAL4-DBD fused PPARgamma LBD expressed in COS7 cells after 24 hrs by luciferase reporter gene assay,B,BAO_0000188,BAO_0000219,Bioorg Med Chem,62.000000,NaN,NaN,NaN,7.21
744,Agonist activity at Gal4-fused human PPARgamma,F,BAO_0000188,BAO_0000019,Bioorg. Med. Chem. Lett.,800.000000,493.903015,8.0,1.0,6.10
619,Agonist activity at human PPARgamma receptor expressed in african green monkey COS cells after 48 hrs by GAL4 transactivation assay,F,BAO_0000188,BAO_0000219,Bioorg. Med. Chem. Lett.,2.000000,528.604980,6.0,1.0,8.70
608,Agonist activity at PPARgamma transfected in HEK293 cells by GAL4 transactivation assay,F,BAO_0000188,BAO_0000219,Bioorg. Med. Chem. Lett.,1100.000000,537.036987,6.0,1.0,5.96
909,Modulation of full-length human pSG5-fused PPARgamma expressed in MG-63 cells co-expressing pGV-P2-PPRE after 24 hrs by luciferase reporter gene based transactivation assay,B,BAO_0000188,BAO_0000219,Eur. J. Med. Chem.,170.000000,581.510010,3.0,2.0,6.77
882,Transactivation of human PPARgamma1 expressed in CHO-K1 cells co-expressing RXRalpha and PPRE after 1 day by luciferase reporter gene assay,B,BAO_0000188,BAO_0000219,Bioorg. Med. Chem.,6.700012,575.708984,4.0,2.0,8.17
1223,Agonist activity at recombinant human GAL4-DBD fused PPARgamma LBD expressed in COS7 cells after 24 hrs by luciferase reporter gene assay,B,BAO_0000188,BAO_0000219,Bioorg Med Chem,550.000000,NaN,NaN,NaN,6.26
460,Agonist activity at human PPARgamma expressed in CV1 cells by receptor transactivation assay,F,BAO_0000188,BAO_0000219,Bioorg. Med. Chem. Lett.,28.000000,391.467010,4.0,1.0,7.55
609,#na#,#na#,#na#,#na#,#na#,NaN,520.968994,6.0,1.0,NaN


# modell

In [33]:
dls = to.dataloaders(bs=64)
learn = tabular_learner(dls, layers=[200, 100], metrics=rmse)
learn.fit_one_cycle(5, 1e-2)

epoch,train_loss,valid_loss,_rmse,time
0,nan,nan,nan,00:00
1,nan,nan,nan,00:00
2,nan,nan,nan,00:00
3,nan,nan,nan,00:00
4,nan,nan,nan,00:00


In [34]:
# Drop rows with missing values
biodata = biodata.dropna()


In [35]:
#
# Create TextDataLoaders for the 'assay_description' column
#text_dls = TextDataLoaders.from_df(data_combined, text_col='assay_description', is_lm=False, valid_pct=0.2)

# Create TabularDataLoaders for the categorical and continuous columns
#procs = [Categorify, Normalize]
#splits = RandomSplitter(valid_pct=0.2)(range_of(data_combined))
#tab_dls = TabularDataLoaders.from_df(data_combined, procs=procs, cat_names=cat_cols, cont_names=cont_cols, y_names='pchembl_value', y_block=RegressionBlock, splits=splits)

# Create a language model using the text_dls
#learn_text = text_classifier_learner(text_dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)
#learn_text = learn_text.load_encoder('finetuned')

# Custom mixed dataloader class
#class MixedDL(DataLoader):
  #  def __init__(self, text_dl, tab_dl):
  #      self.text_dl = text_dl
  #      self.tab_dl = tab_dl
#
  #  def __iter__(self):
  #      for x1, y1 in self.text_dl:
  #          x2, y2 = next(iter(self.tab_dl))
   #         yield (x1, x2), y1

   # def __len__(self):
  #      return len(self.text_dl)

# Combine the TextDataLoaders and TabularDataLoaders using MixedDL
#train_dl = MixedDL(text_dls.train, tab_dls.train)
#valid_dl = MixedDL(text_dls.valid, tab_dls.valid)
#dls_combined = DataLoaders(train_dl, valid_dl)

# Define the model architecture
#class CombinedModel(Module):
   # def __init__(self, text_model, tabular_model):
   #3     self.text_model = text_model
   ##     self.tabular_model = tabular_model
    #    self.final = nn.Linear(100 + 1, 1)

  #  def forward(self, x):
     #   x_text = self.text_model(x[0])
     #   x_tab = self.tabular_model(x[1])
     #   x_combined = torch.cat([x_text, x_tab], dim=1)
     #   return self.final(x_combined)

#text_model = learn_text.model[0]
#tabular_model = tab_dls.model
#model = CombinedModel(text_model, tabular_model)

# Train the model using the combined DataLoader
#learn = Learner(dls_combined, model, loss_func=MSELossFlat(), metrics=rmse)
#learn.fit_one_cycle(5, 1e-2)